# 自定义网络结构

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2649,  0.2354, -0.0767,  0.0657,  0.0566,  0.0585, -0.0829, -0.0446,
         -0.1419, -0.0309],
        [-0.1480,  0.3245, -0.1820, -0.0893,  0.0505,  0.0930, -0.2713,  0.0012,
         -0.1634, -0.0018]], grad_fn=<AddmmBackward0>)

In [2]:
# 第一种自定义的方式：用模型参数声明层
class MLP(nn.Module):
    # 声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[ 0.0243, -0.1028, -0.0970, -0.2484, -0.0287,  0.2762, -0.2246,  0.0129,
         -0.1872, -0.1828],
        [ 0.1045, -0.0394, -0.0938, -0.3644, -0.0595,  0.3756, -0.2444, -0.0706,
         -0.2282, -0.1501]], grad_fn=<AddmmBackward0>)

In [16]:
# 第二种自定义的方式：利用nn._moduleds模块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # `_modules` 中，`block`的类型是OrderedDict。
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [17]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0414, -0.2232, -0.1231, -0.0855, -0.0018, -0.0257,  0.4019, -0.0986,
         -0.0848,  0.0222],
        [ 0.1136, -0.1648, -0.1247, -0.1366, -0.1057, -0.0076,  0.3749, -0.1600,
         -0.0398, -0.1359]], grad_fn=<AddmmBackward0>)

In [18]:
# 下面是对在forward函数中自定义前向传播计算的尝试
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [19]:
net = FixedHiddenMLP()
net(X)

tensor(0.1867, grad_fn=<SumBackward0>)

# 自定义层

In [20]:
# 不带参数的层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()  # 类似的计算可以自己定义

In [21]:
# 带参数的层，nn.Parameter()来进行初始化
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [22]:
# 由于继承了nn.Module，所以可以像使用内置的全连接层一样来访问自定义层
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[5.2504],
        [0.0000]])